In [1]:
import numpy as np
from qiskit import *
from qiskit import IBMQ
import tensorflow as tf
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing
import tensorflow.keras.backend as K
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers, models
from qiskit import QuantumCircuit, transpile
from qiskit.providers.ibmq import least_busy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout
from sympy.utilities.iterables import multiset_permutations

In [2]:
IBMQ.save_account('de404b7eba21afacf3e900bcd1cc1244967cecad8f7051683e08ff88d444aee2e6cb029f51036d2eb8efd6f18bd04acf5d9bc0891891ebe4c1bb1ab062f7741b', overwrite=True)
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [3]:
my_provider = IBMQ.get_provider()
backend  = my_provider.get_backend('ibmq_lima')

In [4]:
x_data0 = np.loadtxt('x_data0.txt', dtype=int)
x_data1 = np.loadtxt('x_data1.txt', dtype=int)
x_data2 = np.loadtxt('x_data2.txt', dtype=int)

y_data0 = np.loadtxt('y_data0.txt', dtype=int)
y_data1 = np.loadtxt('y_data1.txt', dtype=int)
y_data2 = np.loadtxt('y_data2.txt', dtype=int)

In [5]:
print(np.shape(x_data0))
print(np.shape(x_data1))
print(np.shape(x_data2))

(30000, 54)
(30000, 54)
(30000, 54)


In [6]:
print(x_data0[0])
print(x_data1[0])
print(x_data2[0])

[4 3 1 0 2 4 1 0 2 0 4 3 4 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[3 4 0 1 3 1 0 2 2 0 3 2 0 2 1 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [7]:
dim1 = np.shape(x_data0)[1]
dim2 = np.shape(x_data1)[1]
dim3 = np.shape(x_data2)[1]
dimarray = np.array([dim1,dim2,dim3])
print(dimarray)
max_length = np.amax(dimarray)

x_data0 = np.array([np.pad(row, (0, max_length-len(row))) for row in x_data0])
x_data1 = np.array([np.pad(row, (0, max_length-len(row))) for row in x_data1])
x_data2 = np.array([np.pad(row, (0, max_length-len(row))) for row in x_data2])
x_data = np.concatenate((x_data0, x_data1, x_data2))
y_data = np.concatenate((y_data0, y_data1, y_data2))

x_data = preprocessing.normalize(x_data)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)
size = np.shape(x_data)[1]

[54 54 54]


In [46]:
layouts = np.array([0, 1, 2, 3, 4])
k = []
for p in multiset_permutations(layouts):
    k.append(p)
layoutdict = dict(enumerate(k, 0))

encoded = np.empty(np.shape(y_data)[0], dtype=int)
for i in range(np.shape(y_data)[0]):
    for j in range(120):
        if(y_data[i][j] == 1):
            encoded[i] = j

def find_layout_from_encoded(y_pred):
    layouts = np.array([0, 1, 2, 3, 4])
    arr = []
    for p in multiset_permutations(layouts):
        arr.append(p)
    layoutdict = dict(enumerate(arr, 0))
    
    for j in range(120):
        if(y_pred[j] == 1):
            return layoutdict[j]
        
def gatecount(qc, transpiled_circuit):
    count = transpiled_circuit.count_ops()
    items = list(count.items())
    pieces = len(items)
    new_arrays = np.array_split(items, pieces)
    new_arrays
    for i in range(pieces):
        if(new_arrays[i][0][0] == 'cx'):
            cnotcount = int(new_arrays[i][0][1])
            
    return cnotcount
        
def getcircuitfromx(x):
    x = tf.reshape(x, (32, -1, 2))
    dimensions = x.shape
    print(dimensions)
    xprint = K.print_tensor(x[0])
    print(xprint)
    
    qc = QuantumCircuit(5,5)
    qc.cx(x[0,0,0], x[0,0,1])
    
    cnotarray = []
    j = 0
    while(j < dimensions[0]):
        if(x[j] == x[j+1] == 0):
            break
        else:
            cnotdata = x[j:j+2]
            cnotarray.append(cnotdata)
            j = j+2
            
    array = np.array(cnotarray)
    cnotdim = np.shape(array)
    
    qc = QuantumCircuit(5,5)
    for i in range(cnotdim[0]):
        qc.cx(array[i][0], array[i][1])
    
    return qc

#@tf.function
def qiskit_difference(y_true, y_pred, x):
    my_provider = IBMQ.get_provider()
    backend  = my_provider.get_backend('ibmq_lima')
    qc = getcircuitfromx(x)

    layout_true = find_layout_from_encoded(y_true)
    layout_pred = find_layout_from_encoded(y_pred)

    transpiledcircuit_true = transpile(qc, backend, optimization_level=3)
    cnotcount_true = gatecount(qc, transpiledcircuit_true)
    
    transpiledcircuit_pred = transpile(qc, backend, initial_layout=layout_pred)
    cnotcount_pred = gatecount(qc, transpiledcircuit_pred)

    diff = cnotcount_pred - cnotcount_true

    return diff

def custom_loss(x, backend):
    def loss(y_true, y_pred):
        diff = qiskit_difference(y_true=y_true, y_pred=y_pred, x=x, backend=backend)
        return diff    
    return loss

In [47]:
def custom_loss(data, y_pred):
    y_true = data[:, 0:120]
    x = data[:, 120:]
    return qiskit_difference(y_true=y_true, y_pred=y_pred, x=x)

def baseline_model(size):
    i = Input(shape=(size,))
    x = Dense(64, kernel_initializer='glorot_uniform', activation='relu')(i)
    o = Dense(120, kernel_initializer='normal', activation='softmax')(x)
    model = tf.keras.Model(i, o)
    model.compile(loss=custom_loss, optimizer=Adam(learning_rate=0.0005))
    return model

model = baseline_model(size)

In [48]:
model.fit(x_train, np.append(y_train, x_train, axis=1), epochs=90, shuffle=True, verbose=1)

Epoch 1/90
(32, 27, 2)
Tensor("custom_loss/Identity:0", shape=(27, 2), dtype=float32)


StagingError: in user code:

    File "C:\Users\aniru\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\aniru\AppData\Local\Temp/ipykernel_11276/2790879254.py", line 4, in custom_loss  *
        return qiskit_difference(y_true=y_true, y_pred=y_pred, x=x)
    File "C:\Users\aniru\AppData\Local\Temp/ipykernel_11276/1902652884.py", line 69, in qiskit_difference  *
        qc = getcircuitfromx(x)
    File "C:\Users\aniru\AppData\Local\Temp/ipykernel_11276/1902652884.py", line 44, in getcircuitfromx  *
        qc.cx(x[0,0,0], x[0,0,1])
    File "C:\Users\aniru\AppData\Local\Programs\Python\Python39\lib\site-packages\qiskit\circuit\quantumcircuit.py", line 3706, in cx  *
        CXGate(label=label, ctrl_state=ctrl_state), [control_qubit, target_qubit], []
    File "C:\Users\aniru\AppData\Local\Programs\Python\Python39\lib\site-packages\qiskit\circuit\quantumcircui

In [ ]:
#def get_model():
#    i = Input(shape=(size,))
#    x = Dense(64, activation='relu')(i)
#    o = Dense(120, activation='softmax')(x)
#    model = tf.keras.Model(i, o)
#    model.compile(loss=custom_loss(i, backend), optimizer=Adam(learning_rate=0.005), run_eagerly=True)
#    return model

#model = get_model()

In [ ]:
#history = model.fit(x_train, y_train, epochs=100, verbose=True, validation_data=(x_test, y_test))

In [ ]:
model = Sequential()
model.add(Dense(64, input_shape=(size,), activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(120, activation = "softmax"))

opt = Adam(learning_rate=0.01)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs=100, verbose=True, validation_data=(x_test, y_test))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='best')
plt.show()

In [ ]:
newx_data = []
for i in range(np.shape(x_data)[0]):
    newx_data.append(np.reshape(x_data[i], (6, 3, -1)))
newx_data = np.array(newx_data)

x_train, x_test, y_train, y_test = train_test_split(newx_data, y_data, test_size = 0.25)
size = np.shape(x_train[0])

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (2, 2), activation='relu', input_shape=size))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (1, 1), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(120))
          
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs=100, verbose=True, validation_data=(x_test, y_test))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='best')
plt.show()

In [ ]:
#arr = []
#for i in range(np.shape(y_data)[0]):
#    for j in range(120):
#        if(y_data[i][j] == 1):
#            arr.append(j)
#arr = np.array(arr)

In [ ]:
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding
model = Sequential()

# Embedding layer
model.add(
    Embedding(input_dim=num_words,
              input_length = training_length,
              output_dim=100,
              weights=[embedding_matrix],
              trainable=False,
              mask_zero=True))

# Masking layer for pre-trained embeddings
model.add(Masking(mask_value=0.0))

# Recurrent layer
model.add(LSTM(64, return_sequences=False, 
               dropout=0.1, recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])